In [41]:
!pip install pennylane
!pip install pennylane pennylane-lightning[gpu]

### Import Libraries

In [42]:
import numpy as np
import matplotlib.pyplot as plt
import pennylane as qml
from pennylane.templates import QFT
from sklearn.svm import SVC
from sklearn.datasets import fetch_openml, load_digits
from sklearn.preprocessing import MinMaxScaler, normalize
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from pennylane import numpy as pnp
from skimage.transform import resize
from keras.datasets import mnist

### Step 1:  Dataset Preparation


First, we load the MNIST dataset from openML.
- X is the pixel data
- y is the labels
- converting everything to `uint8` here to ensure all values are integers in [0, 255]

In [43]:
# loading mnist from openML
mnist = fetch_openml('mnist_784', version=1, cache=True)
X = mnist['data'].astype(np.uint8) # better to convert for binerization
y = mnist['target'].astype(np.uint8)
y = y.to_numpy()

Next, we focus on 2 classes, i.e. binary classification.
Here, I've been experimenting with different classes, and I stopped on 4 vs 9, cause they have more subtle difference in pixels, they are similar looking.

In [44]:
# focus on binary classification
mask = (y == 4) | (y == 9)
X, y = X[mask], y[mask]
X.shape

(13782, 784)

- I take only the first `n_samples`.
- I convert X to a NumPy array, and shuffling the data randomly

In [45]:
n_samples = 100 # restricting to 6000 samples for now

X = X.values if hasattr(X, "values") else X # safer conversion
perm = np.random.permutation(len(X))
X, y = X[perm], y[perm]

X = X[:n_samples]
y = y[:n_samples]

Now, I normalise pixel intensities.
- [0, 255] -> [0, 1]
- reshaping images back to 2D for resizing, i.e to 28x28 array with float values between 0 and 1.

In [46]:
X = X / 255.0
X = X.reshape(-1, 28, 28)

print(X.shape)
print("Pixel range:", X.min(), X.max())

(100, 28, 28)
Pixel range: 0.0 1.0


And now I reduce images to 8x8 + flattening to (, 64)
- resize -> flatten -> normalize

In [47]:
# convert each 28x28 binarised image to 8x8, then flatten to length 64
def to_8x8_vector(img_row):
    img_8x8 = resize(
        img_row,
        (8, 8),
        anti_aliasing=False,
        preserve_range=True,
        order=1 # controlling interpolation
    )
    img_8x8 = img_8x8.flatten()
    s = np.sum(img_8x8)

    if s > 0:
        img_8x8 = np.sqrt(img_8x8 / s)
    else:
        img_8x8 = np.zeros_like(img_8x8)
        img_8x8[0] = 1.0
      # should be shape (64,)
    return img_8x8

# apply transformation to all images
X_8x8 = np.array([to_8x8_vector(x) for x in X], dtype=float)
X_8x8.shape

(100, 64)

In [48]:
# sanity check, make sure no NaNs exist and all vectors are normalised, i.e. norm is around 1
print("Any NaNs?", np.isnan(X_8x8).any())
print("Norm check:", np.min(np.linalg.norm(X_8x8, axis=1)), np.max(np.linalg.norm(X_8x8, axis=1)))

Any NaNs? False
Norm check: 0.9999999999999999 1.0


I'm gonna do the splitting here, and carry both representations consistently.
- qek inputs: (64,) flattened and normalized vectors, for quantum kernel embedding
- qjpeg: 28x28 images

In [49]:
idx = np.arange(n_samples)

idx_train, idx_test, y_train, y_test = train_test_split(
    idx, y, test_size=0.2, random_state=42, stratify=y
)

# QEK inputs (8x8 -> 64 -> normed)
X_train_qek = X_8x8[idx_train]
X_test_qek  = X_8x8[idx_test]

# QJPEG inputs (28x28 binary images)
X_train_img = X[idx_train]
X_test_img  = X[idx_test]

print("QEK train/test:", X_train_qek.shape, X_test_qek.shape)
print("IMG train/test:", X_train_img.shape, X_test_img.shape)
print("Labels train/test:", y_train.shape, y_test.shape)

QEK train/test: (80, 64) (20, 64)
IMG train/test: (80, 28, 28) (20, 28, 28)
Labels train/test: (80,) (20,)


Data preparation is done.

### Step 2: Quantum Embedding & Kernel Training

Define number of qubits and device.

In [68]:
device = "lightning.gpu"
n_qubits = 6
n_layers = 4          # number of trainable layers
batch_size = 8       # bigger batch for stability
n_steps = 50          # training steps
stepsize = 2e-4       # smaller learning rate
eps = 1e-8      # small epsilon to prevent division by zero
wires = range(n_qubits)

In [69]:
dev = qml.device(device, wires=n_qubits, shots=None)

In [70]:

def qek_layer_amplitude(x, theta_l):
    # Data re-uploading via phase gates (breaks kernel symmetry)
    for q in range(n_qubits):
        qml.RZ(np.pi * x[q], wires=q)

    # Trainable block
    for q in range(n_qubits):
        qml.RX(theta_l[q, 0], wires=q)
        qml.RZ(theta_l[q, 1], wires=q)

    # Entanglement (non-commuting)
    for q in range(n_qubits - 1):
        qml.CNOT(wires=[q, q + 1])

def qek_embedding_amplitude(x, theta):
    qml.AmplitudeEmbedding(
        x,
        wires=range(n_qubits),
        normalize=True
    )

    for l in range(theta.shape[0]):
        qek_layer_amplitude(x, theta[l])

# quantum circuit
@qml.qnode(dev)
def qek_kernel_circuit(x1, x2, theta):
    qek_embedding_amplitude(x1, theta)
    qml.adjoint(qek_embedding_amplitude)(x2, theta)
    return qml.probs(wires=range(n_qubits))

def qek_kernel(x1, x2, theta):
    # Fidelity = probability of |0...0>
    return qek_kernel_circuit(x1, x2, theta)[0]

# compute kernel matrix
def square_kernel_matrix(X, theta):
    n = len(X)
    K = pnp.zeros((n, n))
    for i in range(n):
        for j in range(i, n):
            val = qek_kernel(X[i], X[j], theta)
            K[i, j] = val
            K[j, i] = val
    return K

def rectangular_kernel_matrix(X1, X2, theta):
    K = np.zeros((len(X1), len(X2)))
    for i in range(len(X1)):
        for j in range(len(X2)):
            K[i, j] = qek_kernel(X1[i], X2[j], theta)
    return K

def centered_kernel_alignment(K, y):
    y = y.astype(float)
    y = 2 * y - 1          # {0,1} → {-1,+1}
    yyT = np.outer(y, y)

    Kc = K - K.mean(axis=0) - K.mean(axis=1)[:, None] + K.mean()
    return -np.sum(Kc * yyT) / (np.linalg.norm(Kc) + 1e-8)

def kernel_alignment_loss(theta, X, y, batch_size=10):
    idx = np.random.choice(len(X), batch_size, replace=False)
    Xb = X[idx]
    yb = y[idx]

    K = square_kernel_matrix(Xb, theta)
    return centered_kernel_alignment(K, yb)

def feature_map(x, theta):
    # Amplitude encoding (always normalized)
    qml.AmplitudeEmbedding(x, wires=wires, normalize=False)

    # Variational re-uploading layers
    for l in range(theta.shape[0]):
        for i in range(n_qubits):
            qml.RX(theta[l, i, 0], wires=i)
            qml.RY(theta[l, i, 1], wires=i)
            qml.RZ(theta[l, i, 2], wires=i)

        # Entangling layer (ring or chain)
        for i in range(n_qubits - 1):
            qml.CNOT(wires=[i, i + 1])
        qml.CNOT(wires=[n_qubits - 1, 0])

@qml.qnode(dev, interface="autograd")
def kernel_qnode(x1, x2, theta):
    feature_map(x1, theta)
    qml.adjoint(feature_map)(x2, theta)
    return qml.expval(qml.Projector([0]*n_qubits, wires=wires))

In [71]:
def kernel_matrix(X1, X2, theta):
    return qml.math.stack([
        qml.math.stack([
            kernel_qnode(x1, x2, theta)
            for x2 in X2
        ])
        for x1 in X1
    ])

def kernel_alignment_loss_batch(theta, X, y, batch_size):
    idx = np.random.choice(len(X), batch_size, replace=False)
    Xb = X[idx]
    yb = y[idx]

    # Kernel
    K = kernel_matrix(Xb, Xb, theta)
    K = K / pnp.trace(K)
    K /= pnp.linalg.norm(K)

    # Labels: {0,1} → {−1,+1}
    y_pm = 2 * yb - 1
    yy = qml.math.outer(y_pm, y_pm)

    # Center kernel
    Kc = center_kernel(K)
    Kc = Kc / pnp.trace(Kc)

    # Flatten
    Kf = qml.math.reshape(Kc, (-1,))
    yyf = qml.math.reshape(yy, (-1,))

    # Kernel Target Alignment
    numerator = qml.math.dot(Kf, yyf)
    denominator = qml.math.sqrt(
        qml.math.dot(Kf, Kf) * qml.math.dot(yyf, yyf)
    )

    return -numerator / denominator

def kernel_alignment_loss_minibatch(theta, X, y, batch_pairs=20):
    idx = np.random.choice(len(X), size=batch_pairs, replace=True)

    K_vals = []
    y_vals = []

    for i in idx:
        for j in idx:
            K_vals.append(qek_kernel(X[i], X[j], theta))
            y_vals.append(y[i] * y[j])

    K_vec = pnp.array(K_vals)
    y_vec = pnp.array(y_vals)

    return - pnp.dot(K_vec, y_vec) / (
        pnp.linalg.norm(K_vec) * pnp.linalg.norm(y_vec)
    )

def normalized_kernel_alignment(K, y):
    y = y.reshape(-1, 1)
    Ky = y @ y.T

    K_norm = K / qml.numpy.linalg.norm(K)
    Ky_norm = Ky / qml.numpy.linalg.norm(Ky)

    return 1 - qml.numpy.sum(K_norm * Ky_norm)

def kernel_alignment_loss(theta, X, y):
    K = square_kernel_matrix(X, theta)
    return normalized_kernel_alignment(K, y)

def center_kernel(K):
    n = K.shape[0]
    H = qml.numpy.eye(n) - qml.numpy.ones((n, n)) / n
    return H @ K @ H

In [72]:
print(f"\nTraining expressive circuit with {n_layers} layers")
# Initialize theta with the correct shape (L, n_qubits, 3) for the feature_map
theta = 0.01 * pnp.random.randn(n_layers, n_qubits, 3)

opt = qml.AdamOptimizer(stepsize=stepsize)

for step in range(n_steps):
    # Use kernel_alignment_loss_batch, which is compatible with Autograd
    theta, loss = opt.step_and_cost(
        lambda t: kernel_alignment_loss_batch(t, X_train_qek, y_train, batch_size=batch_size),
        theta
    )
    ema = None
    alpha = 0.9

    ema = loss if ema is None else alpha * ema + (1 - alpha) * loss

    print(f"Step {step:02d} | loss = {loss:.4f} | ema={ema:.3f}")


Training expressive circuit with 4 layers
Step 00 | loss = 18.8774 | ema=18.877
Step 01 | loss = 16.7608 | ema=16.761
Step 02 | loss = 8.5869 | ema=8.587
Step 03 | loss = 13.6563 | ema=13.656
Step 04 | loss = 9.5721 | ema=9.572
Step 05 | loss = 6.1118 | ema=6.112
Step 06 | loss = 17.7762 | ema=17.776
Step 07 | loss = 9.4253 | ema=9.425
Step 08 | loss = 18.4820 | ema=18.482
Step 09 | loss = 16.8951 | ema=16.895
Step 10 | loss = 14.8529 | ema=14.853
Step 11 | loss = 9.1467 | ema=9.147
Step 12 | loss = 10.3433 | ema=10.343
Step 13 | loss = 9.5482 | ema=9.548
Step 14 | loss = 5.6847 | ema=5.685
Step 15 | loss = 12.6881 | ema=12.688
Step 16 | loss = 11.8061 | ema=11.806
Step 17 | loss = 11.3280 | ema=11.328
Step 18 | loss = 14.6396 | ema=14.640
Step 19 | loss = 17.2668 | ema=17.267
Step 20 | loss = 13.9785 | ema=13.979
Step 21 | loss = 11.5302 | ema=11.530
Step 22 | loss = 16.7239 | ema=16.724
Step 23 | loss = 20.1878 | ema=20.188
Step 24 | loss = 12.8252 | ema=12.825
Step 25 | loss = 13.7

### Step 3: QJPEG Compression

In [73]:
def vectorization(img, Cr, Cc, renorm=False):
    "Vectorize the image into amplitude-encoding patches suitable for quantum circuits"
    # splitting the original image (Mr, Mc) into S equal-size patches of shape (Cr, Cc)
    Mr, Mc = img.shape
    assert Mr % Cr == 0 and Mc % Cc == 0
    patches = (img.reshape(Mc//Cr, Cr, -1, Cc).swapaxes(1, 2).reshape(-1, Cr, Cc))
    # 64 patches, (64, 64, 64) shape; S=64

    # vectorize each patch and collect all in a (N, Cr*Cc) array
    vect_patches = np.reshape(patches,  (patches.shape[0], Cr*Cc)) # (64, 4096)

    # normalize each (Cr*Cc) vector to the intensity of the corresponding (Cr, Cc) patch
    states = np.zeros((patches.shape[0], Cr*Cc)) # (64, 4096)
    norm = np.zeros(patches.shape[0])

    for idx in range(patches.shape[0]): # for each patch
        # compute the sum of pixels intensities
        norm[idx] = vect_patches[idx].sum()
        if norm[idx] == 0:
            # empty patch -> encode |0...0>
            states[idx, 0] = 1.0
            norm[idx] = 1.0
            continue

        # normalize the patch vector so that its entries sum is 1
        tmp = vect_patches[idx] / norm[idx]
        # take the element-wise square root of the normalized vector
        states[idx] = np.sqrt(tmp)
    if renorm == False:
        norm = np.ones(patches.shape[0])
    print(states[:10])

    return states, norm # amplitudes, pixel intensities' sums

In [74]:
def qft_swaps(wires):
    n = len(wires)
    # apply QFT to all qubits
    qml.QFT(wires=wires)
    # add swaps to reverse qubit order!
    for i in range(n // 2):
        qml.SWAP(wires=[wires[i], wires[n - i - 1]])


def iqft_swaps(wires):
    n = len(wires)
    # swaps again - BEFORE iqft
    for i in reversed(range(n // 2)):
        qml.SWAP(wires=[wires[i], wires[n-i-1]])
    qml.adjoint(QFT)(wires=wires)

In [75]:
def circuit_builder(states, n0, n2, shots):
    ntilde = (n0 - n2) // 2
    n1 = n0 - ntilde

    qnodes = []

    # define device with n0 qubits
    dev = qml.device(device, wires=n0, shots=shots)

    for idx in range(states.shape[0]):
        # qnode to capture current input state
        @qml.qnode(dev)
        def circuit():
            # print("State norm:", np.linalg.norm(states[idx]))
            # initializing the state (using AmplitudeEmbedding here, but I'm wondering if something else could work faster)
            qml.AmplitudeEmbedding(states[idx], wires=range(n0), normalize=True)

            # Hadamard on all n0 qubits
            for w in range(n0):
                qml.Hadamard(wires=w)

            # apply QFT on all qubits
            qft_swaps(wires=range(n0))

            # apply IQFT on first n1 qubits
            iqft_swaps(wires=range(n1))

            # setting boundaries - Rule 2
            discard_start = n0 // 2 - ntilde
            discard_end = n0 // 2 - 1
            discarded_qubits = set(range(discard_start, discard_end + 1))

            # keep exactly n2 qubits for output
            measured_qubits = list(range(n2))


            # Hadamard on remaining qubits
            for q in measured_qubits:
                qml.Hadamard(wires=q)

            # print(f'Measured qubits: {measured_qubits}')

            return qml.probs(wires=measured_qubits)
        qnodes.append(circuit)

    return qnodes



In [76]:
def reconstruction(qnodes, n2, norm):
    out_freq = np.zeros((len(qnodes), 2**n2))
    for idx, qnode in enumerate(qnodes):
        probs = qnode()
        out_freq[idx] = qnode() * norm[idx]

    return out_freq

In [77]:
def devectorization(out_freq):
    S = out_freq.shape[0]
    nrow = int(np.sqrt(out_freq.shape[1])) # rows per patch
    ncol = nrow

    decoded_patches = np.reshape(out_freq,\
                      (out_freq.shape[0], nrow, ncol)) # (S, nrow, ncol)

    im_h, im_w = nrow*int(np.sqrt(S)), ncol*int(np.sqrt(S)) # final shape

    # initialization
    decoded_img = np.zeros((im_w, im_h))

    idx = 0
    for row in np.arange(im_h - nrow + 1, step=nrow):
        for col in np.arange(im_w - ncol + 1, step=ncol):
            decoded_img[row:row+nrow, col:col+ncol] = decoded_patches[idx]
            idx += 1

    return decoded_img

In [78]:
def qjpeg_feature_map_quantum(img_28x28):
    """
    True QJPEG-inspired feature map:
    - probabilities sum to 1
    - amplitudes = sqrt(probabilities)
    - output dimension = 64 (6 qubits)
    """

    img = img_28x28.astype(float)
    img = img / img.sum()              # probabilities
    amps = np.sqrt(img.flatten())      # amplitudes

    # reduce to 64 amplitudes (simple truncation for now)
    amps = amps[:64]

    # safety
    if np.linalg.norm(amps) == 0:
        amps[0] = 1.0
    else:
        amps /= np.linalg.norm(amps)

    return amps


### Step 4: Inference without retraining

In [79]:
C = 1.0
layers_list = [2, 4, 6, 8, 10, 12]

# --- Devices and QNodes ---
dev = qml.device(device, wires=n_qubits, shots=None)

# Expressive circuit QNode
qnode_expressive = qml.QNode(feature_map, dev)

n_layers_max = max(layers_list)

In [80]:
def combine_qek_qjpeg(x_qek, x_qjpeg):
    """Combine first 32 QEK amplitudes + full QJPEG vector, then normalize."""
    v = np.concatenate([x_qek[:32], x_qjpeg])
    norm = np.linalg.norm(v)
    if norm == 0:
        return v
    return v / norm

def compute_kernel(states_a, states_b=None):
    """Compute quantum kernel matrix from states."""
    if states_b is None:
        states_b = states_a
    K = np.zeros((len(states_a), len(states_b)))
    for i, a in enumerate(states_a):
        for j, b in enumerate(states_b):
            K[i,j] = np.abs(np.vdot(a, b))**2
    return K

def evaluate_kernel(qnode, theta, X_train, X_test, y_train, y_test, C=1.0):
    """Compute kernel, train SVM with precomputed kernel, and return test accuracy."""
    # Compute states
    states_train = np.array([qnode(x, theta) for x in X_train])
    states_test  = np.array([qnode(x, theta) for x in X_test])

    # Kernel computation with safe normalization
    K_train = compute_kernel(states_train)
    max_val = np.max(K_train)
    if max_val == 0:
        max_val = 1.0
    K_train /= max_val

    K_test = compute_kernel(states_test, states_train)
    K_test /= max_val

    # Train SVM
    clf = SVC(kernel="precomputed", C=C)
    clf.fit(K_train, y_train)

    # Predict and return accuracy
    y_pred = clf.predict(K_test)
    return accuracy_score(y_test, y_pred)

def qjpeg_to_32(img_28x28):
    """Return 32-dimensional QJPEG vector from 28x28 image."""
    amps = qjpeg_feature_map_quantum(img_28x28)  # get 64 amplitudes
    amps_32 = amps[:32]  # truncate to 32
    norm = np.linalg.norm(amps_32)
    if norm == 0:
        amps_32[0] = 1.0
        norm = 1.0
    return amps_32 / norm

In [81]:
# --- Prepare QJPEG 32-dimensional vectors ---
X_train_qjpeg = np.array([qjpeg_to_32(img.reshape(28,28)) for img in X_train_img])
X_test_qjpeg  = np.array([qjpeg_to_32(img.reshape(28,28)) for img in X_test_img])

# --- Prepare combined QEK + QJPEG vectors ---
X_train_combined = np.array([
    combine_qek_qjpeg(a, b) for a, b in zip(X_train_qek, X_train_qjpeg)
])
X_test_combined = np.array([
    combine_qek_qjpeg(a, b) for a, b in zip(X_test_qek, X_test_qjpeg)
])

# --- Random initial theta for expressive circuit ---
theta_base = np.random.uniform(
    0, 2*np.pi, size=(max(layers_list), n_qubits, 3)
)

# --- Evaluate circuits ---
results = []


In [84]:
def print_accuracies(layers_count):
    print("\n" + "="*40)
    print(f"Evaluating expressive circuit with {layers_count} layers")
    print("="*40)

    # Define a QNode that returns the state vector
    @qml.qnode(dev)
    def expressive_state_qnode(x, current_theta):
        feature_map(x, current_theta)
        return qml.state()

    # Use theta_base for different numbers of layers, slicing it
    current_theta_params = theta_base[:layers_count]

    # QEK only
    acc_qek = evaluate_kernel(
        expressive_state_qnode,
        current_theta_params,
        X_train_qek,
        X_test_qek,
        y_train,
        y_test,
        C=C
    )
    print(f"QEK accuracy:         {acc_qek:.4f}")

    # QEK + QJPEG combined
    acc_combined = evaluate_kernel(
        expressive_state_qnode,
        current_theta_params,
        X_train_combined,
        X_test_combined,
        y_train,
        y_test,
        C=C
    )
    print(f"QEK + QJPEG accuracy: {acc_combined:.4f}")

    results.append((layers_count, acc_qek, acc_combined))

print_accuracies(n_layers)


Evaluating expressive circuit with 4 layers
QEK accuracy:         0.7500
QEK + QJPEG accuracy: 0.9000
